In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['LAK']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('lymphokine activated killer', 468),
 ('lymphokine activated killer cells', 48),
 ('lymphokine activated killing', 11),
 ('lymphokine il 2 activated killer', 7),
 ('lymphokine activated killer activated', 4),
 ('lymphocytes activated killer', 3),
 ('lym phokine activated killer', 3)]

In [7]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

In [6]:
result = [grounding_map, names, pos_labels]

In [7]:
result

[{'lymphokine activated killer cells': 'MESH:D015979'},
 {'MESH:D015979': 'Killer Cells, Lymphokine-Activated'},
 []]

In [8]:
grounding_map, names, pos_labels = [{'lym phokine activated killer': 'MESH:D015979',
  'lymphocytes activated killer': 'MESH:D015979',
  'lymphokine activated killer': 'MESH:D015979',
  'lymphokine activated killer activated': 'MESH:D015979',
  'lymphokine activated killer cells': 'MESH:D015979',
  'lymphokine activated killing': 'MESH:D015979',
  'lymphokine il 2 activated killer': 'MESH:D015979'},
 {'MESH:D015979': 'Killer Cells, Lymphokine-Activated'},
 ['MESH:D015979']]

In [9]:
excluded_longforms = []

In [10]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [69]:
additional_entities = {}

In [70]:
unambiguous_agent_texts = {'HGNC:20917': ['ALPK1', ['ALPK1']],
                           'HGNC:698': ['ARL4C', ['ARL4C', 'ARL7']]}

In [71]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [72]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [73]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [74]:
intersection1

[]

In [75]:
intersection2

[]

In [76]:
len(corpus)

487

In [77]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [78]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [79]:
names

{'MESH:D015979': 'Killer Cells, Lymphokine-Activated',
 'HGNC:20917': 'ALPK1',
 'HGNC:698': 'ARL4C'}

In [80]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-28 03:28:56] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-28 03:29:00] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9852418216703931 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [81]:
classifier.stats

{'label_distribution': {'MESH:D015979': 487, 'HGNC:20917': 18, 'HGNC:698': 15},
 'f1': {'mean': 0.985242, 'std': 0.012706},
 'precision': {'mean': 0.986771, 'std': 0.011358},
 'recall': {'mean': 0.986538, 'std': 0.011538},
 'HGNC:20917': {'f1': {'mean': 0.864762, 'std': 0.126548},
  'pr': {'mean': 0.783333, 'std': 0.194365},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'HGNC:698': {'f1': {'mean': 0.853333, 'std': 0.129271},
  'pr': {'mean': 0.8, 'std': 0.163299},
  'rc': {'mean': 0.933333, 'std': 0.133333}},
 'MESH:D015979': {'f1': {'mean': 0.993898, 'std': 0.004982},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.987919, 'std': 0.009864}}}

In [82]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [83]:
disamb.dump(model_name, results_path)

In [84]:
print(disamb.info())

Disambiguation model for LAK

Produces the disambiguations:
	ALPK1*	HGNC:20917
	ARL4C*	HGNC:698
	Killer Cells, Lymphokine-Activated*	MESH:D015979

Class level metrics:
--------------------
Grounding                         	Count	F1     
Killer Cells, Lymphokine-Activated*	487	 0.9939
                             ALPK1*	 18	0.86476
                             ARL4C*	 15	0.85333

Weighted Metrics:
-----------------
	F1 score:	0.98524
	Precision:	0.98677
	Recall:		0.98654

* Positive labels
See Docstring for explanation



In [85]:
model_to_s3(disamb)

In [11]:
d = load_disambiguator('LAK')

In [13]:
preds = d.disambiguate(all_texts.values())

In [16]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:698']

In [17]:
len(texts)

0

In [1]:
from adeft.disambiguate import load_disambiguator

In [2]:
ad = load_disambiguator('LAK')

In [3]:
print(ad.info())

Disambiguation model for LAK

Produces the disambiguations:
	ALPK1*	HGNC:20917
	ARL4C*	HGNC:698
	Killer Cells, Lymphokine-Activated*	MESH:D015979

Class level metrics:
--------------------
Grounding                         	Count	F1     
Killer Cells, Lymphokine-Activated*	487	 0.9939
                             ALPK1*	 18	0.86476
                             ARL4C*	 15	0.85333

Global Metrics:
-----------------
	F1 score:	0.98524
	Precision:	0.98677
	Recall:		0.98654

* Positive labels
See Docstring for explanation



In [4]:
ad = load_disambiguator('EAG')

In [5]:
ad.names

{'CHEBI:CHEBI:75882': '1-alkyl-2-acetylglycerol',
 'electroantennogram': 'electroantennogram',
 'FPLX:KCNH': 'KCNH'}

In [6]:
ad.grounding_dict

{'EAG': {'electroantennogram': 'electroantennogram',
  'ether a go go': 'FPLX:KCNH',
  'experimental autoimmune glomerulonephritis': 'ungrounded'}}

In [ ]:
ad